In [1]:
# !pip install catboost
# !pip install datasets
# !pip install transformers

In [2]:
# !cp drive/MyDrive/tomsk_digital_breakthrough.zip ./
# !unzip tomsk_digital_breakthrough.zip

# !mv '╨┤╨░╤В╨░╤Б╨╡╤В 15072022.xlsx' data.xlsx
# !mv "датасет 15072022.xlsx" data.xlsx

In [3]:
from collections import Counter
import numpy as np
import pandas as pd

# from catboost import CatBoostClassifier, Pool

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer

import pickle

In [4]:
df = pd.read_excel("data.xlsx")

In [5]:
# df.head(3)

In [6]:
# for col in ["Коды ТН ВЭД ЕАЭС", "Технические регламенты", "Группа продукции"]:
#     print("---\n"*3)
#     print(col)
#     counter = Counter()
#     for i, row in df.iterrows():
#         tnved_col = row[col]
#         if type(tnved_col) != str:
#             continue
#         tnveds = {t for t in tnved_col.split("; ") if t}
#         for t in tnveds:
#             counter[t] += 1
#     print(counter.most_common(30))
#     print(len(counter))
#     for i in [1, 2, 5, 10, 15, 20, 30, 40, 50, 100]:
#         print(i, len([i for k, v in counter.items() if v >= i]))
#         print(i, ([k for k, v in counter.items() if v == i][:10]))
#     print(list(sorted(counter.keys()))[:20])
#     print(list(sorted(counter.keys()))[-20:])

In [7]:
# df[df["Коды ТН ВЭД ЕАЭС"].astype(str).str.contains("8518")]

In [8]:
# models = dict()
# catboost_params = {
#     'iterations': 500,
#     'learning_rate': 0.1,
#     "loss_function": 'MultiLogloss',
#     # "eval_metric": 'TotalF1:average=Macro',
#     'verbose': 10,
#     # "task_type": "GPU",
#     # "devices": '0',
#     'early_stopping_rounds': 5,
#     'random_seed': 42, 
#     'depth': 6,
#     # "dictionaries": dictionaries
# }

In [9]:
# for TARGET_COL in ["Технические регламенты", "Группа продукции", "Коды ТН ВЭД ЕАЭС"]:
#     tech_regulations_df = df[["Общее наименование продукции", TARGET_COL]]
#     tech_regulations_df = tech_regulations_df.dropna()
#     print(tech_regulations_df.shape)
#     tech_regulations_df = tech_regulations_df.drop_duplicates()
#     tech_regulations_df[TARGET_COL] = tech_regulations_df[TARGET_COL].apply(
#         lambda x: str(x).split("; "))

#     counter = Counter()
#     for i, row in tech_regulations_df.iterrows():
#         tnveds = row[TARGET_COL]
#         for t in tnveds:
#             counter[t] += 1
#     good_cols = {k for k, v in counter.items() if v >= 5}
#     tech_regulations_df = tech_regulations_df[
#         tech_regulations_df[TARGET_COL].apply(
#             lambda x: bool(set(x).intersection(good_cols)))
#     ]
#     print(tech_regulations_df.shape)
#     # Уберем слишком много примеров из самых популярных классов
#     seen_counter = Counter()
#     to_drop_index = []
#     min_val = sorted([v for v in counter.values()])[-5]
#     for i, row in tech_regulations_df.iterrows():
#         tnveds = row[TARGET_COL]
#         for t in tnveds:
#             seen_counter[t] += 1
#         counts = min(seen_counter[t] for t in tnveds)
#         if counts > min_val:
#             to_drop_index.append(i)
#     tech_regulations_df = tech_regulations_df[~tech_regulations_df.index.isin(to_drop_index)]

#     tr_train, tr_val = train_test_split(tech_regulations_df, test_size=0.2, random_state=42)
#     tr_val, tr_test = train_test_split(tr_val, test_size=0.5, random_state=42)

#     mlb = MultiLabelBinarizer()
#     mlb.fit(tr_train[TARGET_COL])
#     models[f"{TARGET_COL}_mlb"] = mlb

#     tfidf = TfidfVectorizer(analyzer="char", ngram_range=(1, 2))
#     tfidf.fit(tr_train["Общее наименование продукции"])
#     models[f"{TARGET_COL}_tfidf"] = tfidf
#     print("tfidf trained")

#     catboost_params["class_names"] = list(mlb.classes_)

#     pools = dict()

#     for dt_name, dt in [("train", tr_train), ("val", tr_val), ("test", tr_test)]:
#         vectors = np.array(
#             tfidf.transform(dt["Общее наименование продукции"]).todense())
#         labels = mlb.transform(dt[TARGET_COL])
#         pools[dt_name] = Pool(vectors, labels)

#     print("pools created")
#     classifier = CatBoostClassifier(**catboost_params)

#     classifier.fit(
#         pools["train"],
#         eval_set=pools["val"],
#         early_stopping_rounds=10,
#         plot=True
#     )
#     models[TARGET_COL] = classifier
#     with open("drive/MyDrive/tomsk_models.pcl", "wb") as f:
#         pickle.dump(models, f)
#     for pool_name, dt in [("val", tr_val), ("test", tr_test)]:
#         preds = classifier.predict(pools[pool_name])
#         for average in ("micro", "macro"):
#             score = f1_score(
#                         mlb.transform(dt[TARGET_COL]),
#                         preds,
#                         average=average)
#             print(pool_name, average, score)
#             models[f"{TARGET_COL}_{pool_name}_{average}"] = score
#     with open("drive/MyDrive/tomsk_models.pcl", "wb") as f:
#         pickle.dump(models, f)

In [10]:
# ls "./Группа продукции_trained"

In [11]:
# !zip -r product_model.zip "./Группа продукции_trained/"

In [12]:
# !cp product_model.zip drive/MyDrive/tomsk_product_model.zip

In [ ]:
from datasets import Dataset
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, 
                          TrainingArguments, Trainer)
from transformers import EarlyStoppingCallback
import torch

models = dict()

def accuracy_thresh(y_pred, y_true, thresh=0.5, sigmoid=True): 
    y_pred = torch.from_numpy(y_pred)
    y_true = torch.from_numpy(y_true)
    if sigmoid: 
      y_pred = y_pred.sigmoid()
    return ((y_pred>thresh)==y_true.bool()).float().mean().item()

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    accuracy = accuracy_score(
        labels,
        (predictions > 0.5).astype(int),
    )
    f1_results = f1_score(labels,
                          (predictions > 0.5).astype(int),
                          average=None)
    f1_micro = f1_score(labels,
                        (predictions > 0.5).astype(int),
                        average="micro")
    f1_macro = f1_score(labels,
                        (predictions > 0.5).astype(int),
                        average="macro")
    
    output = {'accuracy_thresh': accuracy_thresh(predictions, labels),
              'accuracy': accuracy,
              'f1_micro': f1_micro,
              'f1_macro': f1_macro,
              }
    for i, f1 in enumerate(f1_results):
        output[f"f1_{i}"] = f1_results[i]
    return output

def tokenize_function(examples):
    return tokenizer(
        examples["Общее наименование продукции"],
        max_length=128,
        padding='max_length',
        truncation=True)

for TARGET_COL in [
                   "Технические регламенты",
                   "Группа продукции",
                   "Коды ТН ВЭД ЕАЭС_2",
                   "Коды ТН ВЭД ЕАЭС_5",
                   "Коды ТН ВЭД ЕАЭС"
                   ][3:]:
    tnved_slice = None
    if "Коды ТН ВЭД ЕАЭС" in TARGET_COL and TARGET_COL != "Коды ТН ВЭД ЕАЭС":
        tnved_slice = int(TARGET_COL[-1])
        TARGET_COL = "Коды ТН ВЭД ЕАЭС"
    tech_regulations_df = df[["Общее наименование продукции", TARGET_COL]]
    tech_regulations_df = tech_regulations_df.dropna()
    print(tech_regulations_df.shape)
    tech_regulations_df = tech_regulations_df.drop_duplicates()
    tech_regulations_df[TARGET_COL] = tech_regulations_df[TARGET_COL].apply(
        lambda x: str(x).split("; "))
    tech_regulations_df[TARGET_COL] = tech_regulations_df[TARGET_COL].apply(
        lambda x: [tnved.strip() for tnved in x]
    )
    if tnved_slice:
        tech_regulations_df[TARGET_COL] = tech_regulations_df[TARGET_COL].apply(
            lambda x: [tnved[:tnved_slice] for tnved in x]
        )

    counter = Counter()
    for i, row in tech_regulations_df.iterrows():
        tnveds = row[TARGET_COL]
        for t in tnveds:
            counter[t] += 1
    good_cols = {k for k, v in counter.items() if v >= 5}
    tech_regulations_df = tech_regulations_df[
        tech_regulations_df[TARGET_COL].apply(
            lambda x: bool(set(x).intersection(good_cols)))
    ]
    print(tech_regulations_df.shape)
    # Уберем слишком много примеров из самых популярных классов
    seen_counter = Counter()
    to_drop_index = []
    min_val = sorted([v for v in counter.values()])[-5]
    for i, row in tech_regulations_df.iterrows():
        tnveds = row[TARGET_COL]
        for t in tnveds:
            seen_counter[t] += 1
        counts = min(seen_counter[t] for t in tnveds)
        if counts > min_val:
            to_drop_index.append(i)
    tech_regulations_df = tech_regulations_df[~tech_regulations_df.index.isin(to_drop_index)]

    tr_train, tr_val = train_test_split(tech_regulations_df, test_size=0.2, random_state=42)
    tr_val, tr_test = train_test_split(tr_val, test_size=0.5, random_state=42)

    mlb = MultiLabelBinarizer()
    mlb.fit(tr_train[TARGET_COL])
    models[f"{TARGET_COL}_mlb"] = mlb
    with open("tomsk_models_mlb_{TARGET_COL}_{tnved_slice}.pcl", "wb") as f:
        pickle.dump(models, f)
    model_ckpt = "cointegrated/rubert-tiny2"
    tokenizer = AutoTokenizer.from_pretrained(
        model_ckpt, problem_type="multi_label_classification")

    hf_datasets = dict()
    for key, value in [("train", tr_train), ("val", tr_val), ("test", tr_test)]:
        value = value.copy()
        labels = mlb.transform(value[TARGET_COL])
        value["labels"] = list([list(li) for li in labels])
        value = value.drop(TARGET_COL, axis=1)
        dataset = Dataset.from_pandas(value, split="key")
        hf_datasets[key] = dataset.map(tokenize_function, batched=True)


    num_labels = len(mlb.classes_)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_ckpt,
        num_labels=num_labels,
        problem_type="single_label_classification"
    ).to('cuda')

    batch_size = 128

    args = TrainingArguments(
        output_dir=f"multiclass_model_{tnved_slice}",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        metric_for_best_model="f1_macro",
        load_best_model_at_end=True,
        learning_rate=1e-3,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=100,
        weight_decay=0.01,
        save_total_limit=5
    )

    class MultilabelTrainer(Trainer):
        def compute_loss(self, model, inputs, return_outputs=False):
            # print("inputs", inputs.keys())
            labels = inputs.pop("labels")
            outputs = model(**inputs)
            logits = outputs.logits
            loss_fct = torch.nn.BCEWithLogitsLoss()
            loss = loss_fct(logits.view(-1, self.model.config.num_labels), 
                            labels.float().view(-1, self.model.config.num_labels))
            return (loss, outputs) if return_outputs else loss

    multi_trainer = MultilabelTrainer(
        model,
        args,
        train_dataset=hf_datasets["train"],
        eval_dataset=hf_datasets["val"],
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
        callbacks = [EarlyStoppingCallback(early_stopping_patience=10)]
    )

    multi_trainer.evaluate()

    multi_trainer.train()
    folder = f"{TARGET_COL}_trained_{tnved_slice}"
    multi_trainer.save_model(folder)
    !zip -r "{folder}.zip" "./{folder}/"
    

(49327, 2)
(37063, 2)


  0%|          | 0/29 [00:00<?, ?ba/s]

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:875: UserWarning: unknown class(es) ['39191', '84119', '8418', '84388', '85041', '85433', 'XI'] will be ignored
  warnings.warn(


  0%|          | 0/4 [00:00<?, ?ba/s]

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:875: UserWarning: unknown class(es) ['3919', '40', '42023', '49119', '7310', '81089', '83100', '84118', '84388', '84629', '8508', '85433', '90019', '90106', '90330'] will be ignored
  warnings.warn(


  0%|          | 0/4 [00:00<?, ?ba/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not 

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
C:\Users\Admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Общее наименование продукции, __index_level_0__. If Общее наименование продукции, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
C:\Use

Epoch,Training Loss,Validation Loss,Accuracy Thresh,Accuracy,F1 Micro,F1 Macro,F1 0,F1 1,F1 2,F1 3,F1 4,F1 5,F1 6,F1 7,F1 8,F1 9,F1 10,F1 11,F1 12,F1 13,F1 14,F1 15,F1 16,F1 17,F1 18,F1 19,F1 20,F1 21,F1 22,F1 23,F1 24,F1 25,F1 26,F1 27,F1 28,F1 29,F1 30,F1 31,F1 32,F1 33,F1 34,F1 35,F1 36,F1 37,F1 38,F1 39,F1 40,F1 41,F1 42,F1 43,F1 44,F1 45,F1 46,F1 47,F1 48,F1 49,F1 50,F1 51,F1 52,F1 53,F1 54,F1 55,F1 56,F1 57,F1 58,F1 59,F1 60,F1 61,F1 62,F1 63,F1 64,F1 65,F1 66,F1 67,F1 68,F1 69,F1 70,F1 71,F1 72,F1 73,F1 74,F1 75,F1 76,F1 77,F1 78,F1 79,F1 80,F1 81,F1 82,F1 83,F1 84,F1 85,F1 86,F1 87,F1 88,F1 89,F1 90,F1 91,F1 92,F1 93,F1 94,F1 95,F1 96,F1 97,F1 98,F1 99,F1 100,F1 101,F1 102,F1 103,F1 104,F1 105,F1 106,F1 107,F1 108,F1 109,F1 110,F1 111,F1 112,F1 113,F1 114,F1 115,F1 116,F1 117,F1 118,F1 119,F1 120,F1 121,F1 122,F1 123,F1 124,F1 125,F1 126,F1 127,F1 128,F1 129,F1 130,F1 131,F1 132,F1 133,F1 134,F1 135,F1 136,F1 137,F1 138,F1 139,F1 140,F1 141,F1 142,F1 143,F1 144,F1 145,F1 146,F1 147,F1 148,F1 149,F1 150,F1 151,F1 152,F1 153,F1 154,F1 155,F1 156,F1 157,F1 158,F1 159,F1 160,F1 161,F1 162,F1 163,F1 164,F1 165,F1 166,F1 167,F1 168,F1 169,F1 170,F1 171,F1 172,F1 173,F1 174,F1 175,F1 176,F1 177,F1 178,F1 179,F1 180,F1 181,F1 182,F1 183,F1 184,F1 185,F1 186,F1 187,F1 188,F1 189,F1 190,F1 191,F1 192,F1 193,F1 194,F1 195,F1 196,F1 197,F1 198,F1 199,F1 200,F1 201,F1 202,F1 203,F1 204,F1 205,F1 206,F1 207,F1 208,F1 209,F1 210,F1 211,F1 212,F1 213,F1 214,F1 215,F1 216,F1 217,F1 218,F1 219,F1 220,F1 221,F1 222,F1 223,F1 224,F1 225,F1 226,F1 227,F1 228,F1 229,F1 230,F1 231,F1 232,F1 233,F1 234,F1 235,F1 236,F1 237,F1 238,F1 239,F1 240,F1 241,F1 242,F1 243,F1 244,F1 245,F1 246,F1 247,F1 248,F1 249,F1 250,F1 251,F1 252,F1 253,F1 254,F1 255,F1 256,F1 257,F1 258,F1 259,F1 260,F1 261,F1 262,F1 263,F1 264,F1 265,F1 266,F1 267,F1 268,F1 269,F1 270,F1 271,F1 272,F1 273,F1 274,F1 275,F1 276,F1 277,F1 278,F1 279,F1 280,F1 281,F1 282,F1 283,F1 284,F1 285,F1 286,F1 287,F1 288,F1 289,F1 290,F1 291,F1 292,F1 293,F1 294,F1 295,F1 296,F1 297,F1 298,F1 299,F1 300,F1 301,F1 302,F1 303,F1 304,F1 305,F1 306,F1 307,F1 308,F1 309,F1 310,F1 311,F1 312,F1 313,F1 314,F1 315,F1 316,F1 317,F1 318,F1 319,F1 320,F1 321,F1 322,F1 323,F1 324,F1 325,F1 326,F1 327,F1 328,F1 329,F1 330,F1 331,F1 332,F1 333,F1 334,F1 335,F1 336,F1 337,F1 338,F1 339,F1 340,F1 341,F1 342,F1 343,F1 344,F1 345,F1 346,F1 347,F1 348,F1 349,F1 350,F1 351,F1 352,F1 353,F1 354,F1 355,F1 356,F1 357,F1 358,F1 359,F1 360,F1 361,F1 362,F1 363,F1 364,F1 365,F1 366,F1 367,F1 368,F1 369,F1 370,F1 371,F1 372,F1 373,F1 374,F1 375,F1 376,F1 377,F1 378,F1 379,F1 380,F1 381,F1 382,F1 383,F1 384,F1 385,F1 386,F1 387,F1 388,F1 389,F1 390,F1 391,F1 392,F1 393,F1 394,F1 395,F1 396,F1 397,F1 398,F1 399,F1 400,F1 401,F1 402,F1 403,F1 404,F1 405,F1 406,F1 407,F1 408,F1 409,F1 410,F1 411,F1 412,F1 413,F1 414,F1 415,F1 416,F1 417,F1 418,F1 419,F1 420,F1 421,F1 422,F1 423,F1 424,F1 425,F1 426,F1 427,F1 428,F1 429,F1 430,F1 431,F1 432,F1 433,F1 434,F1 435,F1 436,F1 437,F1 438,F1 439,F1 440,F1 441,F1 442,F1 443,F1 444,F1 445,F1 446,F1 447,F1 448,F1 449,F1 450,F1 451,F1 452,F1 453,F1 454,F1 455,F1 456,F1 457,F1 458,F1 459,F1 460,F1 461,F1 462,F1 463,F1 464,F1 465,F1 466,F1 467,F1 468,F1 469,F1 470,F1 471,F1 472,F1 473,F1 474,F1 475,F1 476,F1 477,F1 478,F1 479,F1 480,F1 481,F1 482,F1 483,F1 484,F1 485,F1 486,F1 487,F1 488,F1 489,F1 490,F1 491,F1 492,F1 493,F1 494,F1 495,F1 496,F1 497,F1 498,F1 499,F1 500,F1 501,F1 502,F1 503,F1 504,F1 505,F1 506,F1 507,F1 508,F1 509,F1 510,F1 511,F1 512,F1 513,F1 514,F1 515,F1 516,F1 517,F1 518,F1 519,F1 520,F1 521,F1 522,F1 523,F1 524,F1 525,F1 526,F1 527,F1 528,F1 529,F1 530,F1 531,F1 532,F1 533,F1 534,F1 535,F1 536,F1 537,F1 538,F1 539,F1 540,F1 541,F1 542,F1 543,F1 544,F1 545,F1 546,F1 547,F1 548,F1 549,F1 550,F1 551,F1 552,F1 553,F1 554,F1 555,F1 556,F1 557,F1 558,F1 559,F1 560,F1 561,F1 562,F1 563,F1 564,F1 565,F1 566,F1 567,F1 568,F1 569,F1 570,F1 571,F1 572,F1 573,F1 574,F1 575

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Общее наименование продукции, __index_level_0__. If Общее наименование продукции, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3545
  Batch size = 128
C:\Users\Admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
C:\Users\Admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _w

In [ ]:
# dictionaries = [
#     {
#         'dictionary_id': 'Unigram',
#         'max_dictionary_size': '50000',
#         'gram_count': '1',
#     },
#     {
#         'dictionary_id': 'Bigram',
#         'max_dictionary_size': '50000',
#         'gram_count': '2',
#     }
# ]




In [ ]:
# pools = dict()

# for dt_name, dt in [("train", tr_train), ("val", tr_val), ("test", tr_test)]:
#     text_dt = dt.drop("Технические регламенты", axis=1)
#     pools[dt_name] = Pool(
#         text_dt,
#         mlb.transform(dt["Технические регламенты"]),
#         text_features=["Общее наименование продукции"],
#     ) 
# print("pools created")
# # classifier = CatBoostClassifier(**catboost_params)
# classifier = CatBoostClassifier(
#     loss_function='MultiLogloss',
#     # eval_metric='HammingLoss',
#     iterations=500,
#     class_names=list(mlb.classes_)
# )

# # train_pool = Pool(dt[["Общее наименование продукции"]].reset_index().drop("index", axis=1)[:375], Y_train, text_features=["Общее наименование продукции"])
# train_pool = Pool(tr_train.drop("Технические регламенты", axis=1), mlb.transform(tr_train["Технические регламенты"]), text_features=["Общее наименование продукции"])
# # test_pool = Pool(X_test, Y_test)
# # classifier.fit(train_pool, metric_period=10, plot=True, verbose=50)
# classifier.fit(
#     # pools["train"],
#     # eval_set=pools["val"],
#     train_pool,
#     early_stopping_rounds=10,
#     metric_period=10,
#     plot=True
# )

# # clf = CatBoostClassifier(
# #     loss_function='MultiLogloss',
# #     eval_metric='HammingLoss',
# #     iterations=500,
# #     class_names=['A', 'B', 'C', 'D', 'E']
# # )
# # clf.fit(train_pool, metric_period=10, plot=True, verbose=50)

# for pool_name, dt in [("val", tr_val), ("test", tr_test)]:
#     preds = classifier.predict(pools[pool_name])
#     for average in ("micro", "macro"):
#         print(
#             average,
#             f1_score(dt["Технические регламенты"],
#                      preds,
#                      average=average)
#         )

In [ ]:
# from catboost import CatBoostClassifier, Pool
# from catboost.utils import eval_metric
# from sklearn.datasets import make_multilabel_classification
# from sklearn.model_selection import train_test_split

# X, Y = make_multilabel_classification(n_samples=500, n_features=20, n_classes=5, random_state=0)
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
# train_pool = Pool(X_train, Y_train)
# test_pool = Pool(X_test, Y_test)

# clf = CatBoostClassifier(
#     loss_function='MultiLogloss',
#     eval_metric='HammingLoss',
#     iterations=500,
#     class_names=['A', 'B', 'C', 'D', 'E']
# )
# clf.fit(train_pool, eval_set=test_pool, metric_period=10, plot=True, verbose=50)